<a href="https://www.kaggle.com/code/datascientistsalman/unlocking-magnus-carelson-chess-playing-strategies?scriptVersionId=210695865" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**Created by:** Salman Ahmed  
**LinkedIn:** [Salman Ahmed ](https://www.linkedin.com/in/salmanahmed84/)  
**Portfolio:** [Salman's Portfolio](https://github.com/salman-datascientist)

**Gemini 1.5 with Long-Context Windows for Analyzing Hours-Long Magnus Carlsen Chess Game Videos**

***Extended Context Length:***
Vast Input Capacity: Gemini 1.5 Pro excels at processing hours-long chess game videos, managing inputs of up to 10 million tokens. This allows for seamless analysis of entire tournaments without segmenting the data into smaller chunks​.
Improved Contextual Understanding: The extended context length enables the model to connect strategic decisions across games, identify long-term patterns, and correlate player behavior over time.

***High Recall:***
Accurate Information Retrieval: When analyzing hours of video, Gemini 1.5 Pro can precisely locate specific moves, strategies, or key moments, such as a player executing a checkmate or pivotal errors in gameplay. This high recall ensures no critical detail is missed​.
Superior Performance: Compared to other models, Gemini 1.5 Pro demonstrates unmatched accuracy in retrieving and reasoning over long video sequences, even outperforming specialized models like GPT-4V for video QA.
Multimodal Capabilities:

***Unified Understanding:***
Gemini 1.5 Pro integrates video, audio commentary, and text-based annotations in chess games to create a comprehensive analysis of gameplay strategies and decisions.
Enhanced Contextual Awareness: By combining visual inputs (e.g., board state), audio (e.g., player commentary), and text (e.g., move notations), the model delivers richer insights into player strategies and decision-making processes.
Efficiency and Scalability:

***Optimized Processing:*** 
Designed to handle extensive data, Gemini 1.5 Pro efficiently processes multiple hours of chess gameplay videos, ensuring that even tournaments spanning several days can be analyzed in one session.
Scalability: Its ability to analyze entire archives of chess games, such as championship series, makes it ideal for researchers, commentators, or AI systems studying gameplay evolution and strategy trends​.
Reduced Reliance on External Tools:

***Self-Sufficiency:***
Unlike systems that rely on external video processing or annotation tools, Gemini 1.5 Pro can directly analyze video content, extracting timestamps, moves, and strategic insights without additional preprocessing​.

***Simplified Workflow:***
This all-in-one capability simplifies the workflow, enabling faster and more efficient analysis of chess games without requiring integration with separate retrieval or indexing systems​.

Install the SDK

In [20]:
%pip install -U -q "google-generativeai>=0.8.3"

Note: you may need to restart the kernel to use updated packages.


<center>
  <h2 style="color: blue;">YouTube Video: <a href="https://www.youtube.com/watch?v=RkBgf06G89E" target="_blank" style="color: blue;">Magnus Carlsen goes 94% BERSERK in Blitz Titled Arena</a></h2>
</center>

In [21]:
from IPython.display import HTML

# Display a centered YouTube video
video_id = "RkBgf06G89E"
HTML(f"""
<div style="display: flex; justify-content: center;">
    <iframe width="1024" height="576" src="https://www.youtube.com/embed/{video_id}" frameborder="0" allowfullscreen></iframe>
</div>
""")

In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemini-long-context/submission_instructions.txt
/kaggle/input/chess-guides/Chess_ The Complete Guide To Chess - Master_ Chess Tactics Chess Openings and Chess Strategies.pdf
/kaggle/input/chess-guides/1 Chess Game Instructions.pdf
/kaggle/input/magnus-chess-blitz/magnus_chess_blitz.mp4


In [23]:
import os
import time
import datetime
import google.generativeai as genai
from google.generativeai import caching
from IPython.display import HTML, Markdown, display, Video

Get GEMINI_API_KEY stored in User Secrets.

In [24]:
# Get the API key from here: https://ai.google.dev/tutorials/setup
# Create a new secret called "GEMINI_API_KEY" via Add-ons -> Secrets in the top menu, and attach it to this notebook.

from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


**Choice of Gemini Model:**

gemini-1.5-flash, alias that points to gemini-1.5-flash-002

* Best for: Image understanding, Video understanding, Audio understanding
* Use case: Process 3,000 images at a time, Look through 1 hour long videos, Listen to hours of audio
ed here.

!Testing ! - Run your first prompt
In this step, you will test that your API key is set up correctly by making a request. The gemini-1.5-flash model has been selected here.

In [35]:
# found 'gemini-1.5-flash-001' response to be vastly different and more useful, but still went with latest version of gemini flash
#flash = genai.GenerativeModel('gemini-1.5-flash-001')
# Create the model
generation_config = {
    "temperature": 0.7,
    "top_p": 0.75,
    "top_k": 10,
    "max_output_tokens": 8192
}


flash = genai.GenerativeModel(model_name='gemini-1.5-flash-002', generation_config=generation_config)
response = flash.generate_content("Explain AI to me like I'm a kid.")
print(response.text)

Imagine you have a super smart puppy.  You teach it tricks, like "sit" and "fetch".  At first, it's clumsy, but the more you teach it and show it examples, the better it gets!

Artificial Intelligence, or AI, is like that super smart puppy, but instead of learning tricks, it learns from information we give it.  We feed it lots and lots of pictures of cats, for example, and it learns what a cat looks like.  Then, if you show it a new picture, it can guess if it's a cat or not!

AI can do lots of other things too, like:

* **Play games:**  Some AIs are really good at chess or video games!
* **Help you find things:**  Like when you ask Google a question, AI helps it find the right answer.
* **Translate languages:**  AI can change words from one language to another.
* **Draw pictures:**  Some AIs can even create amazing artwork!

It's still learning and getting better all the time, just like your puppy!  It's not really *thinking* like a person, but it's getting really good at doing things

***When to use context caching***

Context caching is particularly well suited to scenarios where a substantial initial context is referenced repeatedly by shorter requests. Consider using context caching for use cases such as:
* Chatbots with extensive system instructions
* Repetitive analysis of lengthy video files
* Recurring queries against large document sets
* Frequent code repository analysis or bug fixingtput tokens.

***How caching reduces costs***

Context caching is a paid feature designed to reduce overall operational costs. Billing is based on the following factors:

1. **Cache token count**: The number of input tokens cached, billed at a reduced rate when included in subsequent prompts.
2. **Storage duration**: The amount of time cached tokens are stored (TTL), billed based on the TTL duration of cached token count. There are no minimum or maximum bounds on the TTL.
3. **Other factors**: Other charges apply, such as for non-cached input tokens and output tokens.


In [ ]:
magnus_video1 = '/kaggle/input/magnus-chess-blitz/magnus_chess_blitz.mp4'
Video(magnus_video1, embed=True, width=640, height=480)

In [ ]:
chess_game_instructions_1 = '/kaggle/input/chess-guides/1 Chess Game Instructions.pdf'

In [ ]:
chess_game_instructions_2 = '/kaggle/input/chess-guides/Chess_ The Complete Guide To Chess - Master_ Chess Tactics Chess Openings and Chess Strategies.pdf'

**Clearing Cached Content**

Cllear any existing cached content to ensure a fresh start.

In [36]:
# Clear cached content to avoid using old data

for content in genai.caching.CachedContent.list():

    genai.caching.CachedContent.delete(content)

In [37]:
# Upload the video using the Files API
video_file = genai.upload_file(path=magnus_video1)

# Wait for the file to finish processing
while video_file.state.name == 'PROCESSING':
  print('Waiting for video to be processed.')
  time.sleep(2)
  video_file = genai.get_file(video_file.name)

print(f'Video processing complete: {video_file.uri}')


Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to

In [39]:
pdf_file_1 = genai.upload_file(path=chess_game_instructions_1)

pdf_file_2 = genai.upload_file(path=chess_game_instructions_2)

In [40]:

# Create a cache with a 60 minute TTL
cache = caching.CachedContent.create(
    model='models/gemini-1.5-flash-002',
    display_name='magnus chess blitz of 51 games', # used to identify the cache
    system_instruction=(
        'You are an AI expert chess games video analyst and audio commentator analyst for player Magnus Carlsen (DrNykterstein) who plays 51 blitz chess games, and your job is to answer '
        'the user\'s query based on the video file you have access to.'
        'You can read the two pdf_files to understand how chess is played and the various strategies used for winning'
        'Here is an interview of Magnus Carlsen (DrNykterstein) explaining how he plays'
        'I don\'t calculate much or try looking for the right move.'
        'I always say that i can feel the correct move i.e. intuition.'
        'Many people have been asking how I trained my intuition to become so powerful that I don\'t need to look for the right moves, but rather feel it.'
        'Well here is my secret. I have memorised over 10,000 great games and to this very day I still remember it.'
        'I dont just memorise the games at first. I analyse them, make my own notes, sleep on my notes, and sometimes use an engine to assist me.'
        'I memorise the game to the point that I have dreams of it.'
        'When I don\'t have a chess board in front of me i visualize it and replay the game in my head and along side it i make make more mental notes or futher improve on it.'
        'Not only does this help me in blindfold chess but it also makes it a whole hell of a lot easier to calculate.'
        'I repeat these steps until I can literally play the game without having to concentrate on moves.'
        'I do this for every great game that has been played.'
        'Now you might be asking how does this relate to my super strong unrivaled intuition.'
        'It\'s easy. Seeing as I have flawlessly memorised 10,000 games the correct moves are always in the back of my head.'
        'If in a game I am in an uncharted territory (which is rare) I will replay a similar game to the one I\'m playing at the board and get ideas from it.'
        'Having all of these moves memorised makes it much easier for me to recognise the correct move.'
        'Thus practicing this skill took my intuition on a whole another level.'
        'It\'s like I\'m cheating because I have access to my own database in my head.' 
        'Hahaha but I\'m not technically cheating. Hard work always pays off.'
        'So my advice to you is to; analyse, annotate, memorise every great game played. Sleep on the games, read your notes the first thing in the morning, play the games in your head (whenever you\'re bored i.e. on the bus etc.)'
        'And always hit the gym. Your physical health is just as important as your mental health.'
        'Sometimes when two powerful opponents clash off on the board the end result is determined by the one who is in better shape and can keep calculating.'
        ),
    contents=[video_file, pdf_file_1, pdf_file_2],
    ttl=datetime.timedelta(minutes=60),
)

# Create the model
generation_config = {
    "temperature": 0.7,
    "top_p": 0.75,
    "top_k": 10,
    "max_output_tokens": 8192
}

# Construct a GenerativeModel which uses the created cache.
#model = genai.GenerativeModel.from_cached_content(cached_content=cache, generation_config=generation_config)
model = genai.GenerativeModel.from_cached_content(cached_content=cache, generation_config=generation_config)

In [41]:
# Query the model
response = model.generate_content([(
    'I am learning chess. Can you summarize the strategies from watching DrNykterstein (aka Magnus Carlsen) moves')])

print(response.usage_metadata)

# The output should look something like this:
#
# prompt_token_count: 696219
# cached_content_token_count: 696190
# candidates_token_count: 214
# total_token_count: 696433

prompt_token_count: 3010505
candidates_token_count: 520
total_token_count: 3011025
cached_content_token_count: 3010480



In [42]:
print(response.text)

Analyzing Magnus Carlsen's games to distill overarching strategies is difficult because his style is so versatile and adaptable.  He doesn't rigidly adhere to one specific opening or middlegame plan. However, some common threads emerge from observing his play:

**1. Proactive Development and Piece Coordination:** Carlsen prioritizes rapid, harmonious development of his pieces. He rarely makes passive moves; each move contributes to a plan, often aiming for a central presence and good piece coordination. He's masterful at finding the best squares for his pieces, often subtly improving their positions over several moves.

**2. Subtle Strategic Advantage:** He excels at creating small, incremental advantages that accumulate over the course of the game. This might involve a slightly better pawn structure, a more active piece placement, or a superior king safety.  He doesn't necessarily go for flashy tactical sacrifices, but rather builds a position where his opponent is gradually squeezed.

Define methods to upload large data/video files.

In [43]:
# Query the model
response = model.generate_content([(
    'Refine your answer and summarize the Pawn Structure strategy from watching DrNykterstein (aka Magnus Carlsen) moves in game1 as black, game 2 as white, game 3 as white')])

print(response.usage_metadata)

# The output should look something like this:
#
# prompt_token_count: 696219
# cached_content_token_count: 696190
# candidates_token_count: 214
# total_token_count: 696433

prompt_token_count: 3010522
candidates_token_count: 592
total_token_count: 3011114
cached_content_token_count: 3010480



In [44]:
print(response.text)

Analyzing DrNykterstein's (Magnus Carlsen's) games to extract a single, overarching "Pawn Structure Strategy" is difficult because his approach is highly nuanced and context-dependent.  He doesn't rigidly adhere to one specific pawn structure strategy across games. However, we can identify recurring themes in his pawn play across three hypothetical games:

**General Themes in DrNykterstein's Pawn Play (Hypothetical Games):**

* **Flexibility and Adaptability:**  His pawn moves are rarely pre-planned for a specific endgame or structure. Instead, they support his overall strategic plan, often aiming for slight imbalances and dynamic play.  He's less focused on achieving a particular pawn structure and more on creating weaknesses in the opponent's position.

* **Prophylaxis:** He pays close attention to preventing opponent's pawn breaks and plans. His pawn moves often serve to restrict opponent's options or prepare for future maneuvering.

* **Centralization and Control:**  While not alwa

In [45]:
# Query the model
response = model.generate_content([(
    'Refine your answer and summarize the best defense strategy against DrNykterstein (aka Magnus Carlsen) from watching DrNykterstein (aka Magnus Carlsen) moves')])

print(response.usage_metadata)

# The output should look something like this:
#
# prompt_token_count: 696219
# cached_content_token_count: 696190
# candidates_token_count: 214
# total_token_count: 696433

prompt_token_count: 3010517
candidates_token_count: 421
total_token_count: 3010938
cached_content_token_count: 3010480



In [46]:
print(response.text)

Observing Magnus Carlsen's games reveals no single "best" defense, as his style is incredibly adaptable and his strength lies in exploiting weaknesses rather than relying on specific opening traps.  However, a strong defense against him hinges on these principles:

* **Solid Preparation:**  Carlsen is exceptionally well-prepared.  Facing him unprepared is a recipe for disaster.  Thorough opening preparation, extending beyond the main lines into variations he might employ, is crucial.  Focus on understanding the underlying strategic ideas rather than memorizing moves.

* **Accurate Calculation:** Carlsen excels at precise calculation.  Avoid tactical blunders, which he will ruthlessly punish.  Double-check your calculations, especially in complex positions.

* **Prophylaxis:** Anticipate his plans and prevent them before they develop.  Don't just react; actively shape the position to your advantage and limit his options.

* **Strategic Understanding:**  Don't get bogged down in tactical

**List caches**

It's not possible to retrieve or view cached content, but you can retrieve cache metadata (name, model, display_name, usage_metadata, create_time, update_time, and expire_time).

To list metadata for all uploaded caches, use CachedContent.list():

In [ ]:
for c in caching.CachedContent.list():
  print(c)

**Update a cache**


You can set a new ttl or expire_time for a cache. Changing anything else about the cache isn't supported.

The following example shows how to update the ttl of a cache using CachedContent.update().

In [ ]:
cache.update(ttl=datetime.timedelta(hours=2))

**Delete a cache**

The caching service provides a delete operation for manually removing content from the cache. The following example shows how to delete a cache using CachedContent.delete().

In [ ]:
cache.delete()

## Bibliography
* https://aistudio.google.com/app/apikey
* https://storage.googleapis.com/deepmind-media/gemini/gemini_v1_5_report.pdf
* https://ai.google.dev/gemini-api/docs/text-generation?lang=python
* https://ai.google.dev/api/generate-content#image